In [63]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', None)

In [64]:
###Read in worksheets

xls = pd.ExcelFile('raw_data_for_cleaning.xlsx')
transac = pd.read_excel(xls, 'Transactions')
custdemo = pd.read_excel(xls, 'CustomerDemographic')
custadd = pd.read_excel(xls, 'CustomerAddress')

In [65]:
transac.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [66]:
transac.shape


(20000, 13)

In [67]:
transac.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date'],
      dtype='object')

In [68]:
###Identify records with null values
transac.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [69]:
###Identify proportion of records with null values
percent_missing = transac.isnull().sum() * 100 / len(transac)
missing_value_transac = pd.DataFrame({'percent_missing': percent_missing})
missing_value_transac.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_transac

,percent_missing
online_order,1.800
brand,0.985
product_line,0.985
product_class,0.985
product_size,0.985
standard_cost,0.985
product_first_sold_date,0.985
transaction_id,0.000
product_id,0.000
customer_id,0.000


In [70]:
transac['transaction_id'].nunique()

20000

In [71]:
transac['customer_id'].nunique()

3494

In [72]:
###List values and counts per column
for c in transac.columns:
    print ("---- %s ---" % c)
    print (transac[c].value_counts())

---- transaction_id ---
2049     1
17714    1
15677    1
13628    1
3387     1
        ..
4759     1
6806     1
661      1
2708     1
2047     1
Name: transaction_id, Length: 20000, dtype: int64
---- product_id ---
0      1378
3       354
1       311
35      268
38      267
       ... 
71      137
16      136
8       136
100     130
47      121
Name: product_id, Length: 101, dtype: int64
---- customer_id ---
1068    14
2476    14
2183    14
2548    13
3232    13
        ..
2876     1
822      1
2477     1
1846     1
1544     1
Name: customer_id, Length: 3494, dtype: int64
---- transaction_date ---
2017-08-18    82
2017-02-14    82
2017-10-15    76
2017-01-31    73
2017-12-19    71
              ..
2017-01-12    38
2017-12-07    37
2017-03-29    36
2017-09-25    35
2017-10-19    32
Name: transaction_date, Length: 364, dtype: int64
---- online_order ---
1.0    9829
0.0    9811
Name: online_order, dtype: int64
---- order_status ---
Approved     19821
Cancelled      179
Name: order_status,

In [73]:
###Drop NAs and Nulls
clean_transac = transac.dropna(how='any',axis=0)

In [74]:
clean_transac.shape

(19445, 13)

In [75]:
clean_transac.isnull().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

In [76]:
clean_transac.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object

In [77]:
###Convert product_first_sold_date to date format
clean_transac['new_product_first_sold_date'] = pd.to_timedelta(clean_transac.product_first_sold_date, unit='D') + pd.to_datetime('1899-12-30')

<ipython-input-77-65dacac34c48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_transac['new_product_first_sold_date'] = pd.to_timedelta(clean_transac.product_first_sold_date, unit='D') + pd.to_datetime('1899-12-30')


In [78]:
###Drop old date column
clean_transac = clean_transac.drop('product_first_sold_date', 1)

In [79]:
clean_transac.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [80]:
clean_transac.transaction_date.max()

Timestamp('2017-12-30 00:00:00')

In [81]:
custdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [82]:
custdemo.shape

(4000, 13)

In [83]:
custdemo['customer_id'].nunique()

4000

In [84]:
custdemo.columns

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object')

In [85]:
###Identify records with null values
custdemo.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

In [86]:
for c in custdemo.columns:
    print ("---- %s ---" % c)
    print (custdemo[c].value_counts())

---- customer_id ---
2049    1
3391    1
1318    1
3367    1
1322    1
       ..
677     1
2728    1
681     1
2732    1
2047    1
Name: customer_id, Length: 4000, dtype: int64
---- first_name ---
Timmie      5
Max         5
Tobe        5
Kippy       4
Kim         4
           ..
Domeniga    1
Teriann     1
Belia       1
Mata        1
Antonino    1
Name: first_name, Length: 3139, dtype: int64
---- last_name ---
Ramsdell     3
Pristnor     3
Lithgow      2
Struys       2
Mungham      2
            ..
Wallis       1
Scedall      1
Waleran      1
Braundt      1
Kitchener    1
Name: last_name, Length: 3725, dtype: int64
---- gender ---
Female    2037
Male      1872
U           88
M            1
F            1
Femal        1
Name: gender, dtype: int64
---- past_3_years_bike_related_purchases ---
19    56
16    56
67    54
20    54
2     50
      ..
8     28
85    27
86    27
95    27
92    24
Name: past_3_years_bike_related_purchases, Length: 100, dtype: int64
---- DOB ---
1978-01-30    7
1

In [87]:
###Identify proportion of records with null values
percent_missing = custdemo.isnull().sum() * 100 / len(custdemo)
missing_value_custdemo = pd.DataFrame({'percent_missing': percent_missing})
missing_value_custdemo.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_custdemo

,percent_missing
job_industry_category,16.400
job_title,12.650
default,7.550
last_name,3.125
DOB,2.175
tenure,2.175
customer_id,0.000
first_name,0.000
gender,0.000
past_3_years_bike_related_purchases,0.000


In [88]:
###Drop NAs and Nulls
clean_custdemo = custdemo.dropna(how='any',axis=0)
###Drop 'default' column
clean_custdemo = clean_custdemo.drop('default', 1)
clean_custdemo.shape

(2630, 12)

In [89]:
clean_custdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
8,9,Mala,Lind,Female,97,1973-03-10,Business Systems Development Analyst,Argiculture,Affluent Customer,N,Yes,8.0
9,10,Fiorenze,Birdall,Female,49,1988-10-11,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,20.0


In [90]:
###Convert 'gender' values to uniform format
clean_custdemo['gender'] = clean_custdemo['gender'].replace('Femal' ,'Female')

In [91]:
clean_custdemo['gender'] = clean_custdemo['gender'].replace('F' ,'Female')

In [92]:
clean_custdemo['gender'].value_counts()

Female    1368
Male      1262
Name: gender, dtype: int64

In [93]:
latest_transaction = dt.datetime(2017,12,30)

In [94]:
###Create customer age column
clean_custdemo['age'] = latest_transaction - clean_custdemo.DOB

clean_custdemo['age'] = clean_custdemo.age / np.timedelta64(1, 'Y')

clean_custdemo['age'] = clean_custdemo['age'].astype(np.int64)

clean_custdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,37
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,63
8,9,Mala,Lind,Female,97,1973-03-10,Business Systems Development Analyst,Argiculture,Affluent Customer,N,Yes,8.0,44
9,10,Fiorenze,Birdall,Female,49,1988-10-11,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,20.0,29


In [95]:
for c in clean_custdemo.columns:
    print ("---- %s ---" % c)
    print (clean_custdemo[c].value_counts())

---- customer_id ---
2049    1
2686    1
2688    1
641     1
2690    1
       ..
3393    1
1346    1
1350    1
3399    1
2047    1
Name: customer_id, Length: 2630, dtype: int64
---- first_name ---
Tobe        5
Gerome      4
Osgood      4
Lindsay     4
Adrianne    3
           ..
Eli         1
Farrel      1
Jammal      1
Maren       1
Hayden      1
Name: first_name, Length: 2234, dtype: int64
---- last_name ---
Pristnor       2
Antognazzi     2
Dwerryhouse    2
Pountain       2
Zisneros       2
              ..
Odlin          1
Jerche         1
Brown          1
McIlhone       1
Orehead        1
Name: last_name, Length: 2560, dtype: int64
---- gender ---
Female    1368
Male      1262
Name: gender, dtype: int64
---- past_3_years_bike_related_purchases ---
2     39
73    39
38    37
48    34
97    34
      ..
8     19
1     18
21    18
58    15
92    14
Name: past_3_years_bike_related_purchases, Length: 100, dtype: int64
---- DOB ---
1978-01-30    6
1976-07-16    4
1999-07-28    3
1988-11

In [96]:
custadd.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [97]:
custadd.shape

(3999, 6)

In [98]:
custadd.columns

Index(['customer_id', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')

In [99]:
###Identify records with null values


custadd.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [100]:
custadd['customer_id'].nunique()

3999

In [101]:
for c in custadd.columns:
    print ("---- %s ---" % c)
    print (custadd[c].value_counts())

---- customer_id ---
2049    1
1346    1
3367    1
1322    1
3371    1
       ..
2724    1
677     1
2728    1
681     1
2047    1
Name: customer_id, Length: 3999, dtype: int64
---- address ---
3 Talisman Place           2
3 Mariners Cove Terrace    2
64 Macpherson Junction     2
60447 Ridgeway Plaza       1
25 Graceland Way           1
                          ..
572 Spohn Drive            1
08180 Golf Park            1
95294 Karstens Park        1
7380 Canary Pass           1
1 Namekagon Parkway        1
Name: address, Length: 3996, dtype: int64
---- postcode ---
2170    31
2145    30
2155    30
2153    29
3977    26
        ..
3331     1
3036     1
3321     1
3305     1
2143     1
Name: postcode, Length: 873, dtype: int64
---- state ---
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64
---- country ---
Australia    3999
Name: country, dtype: int64
---- property_valuation ---
9     647
8 

In [102]:
clean_custadd = custadd
clean_custadd.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [103]:
###Convert 'gender' values to uniform format
clean_custadd['state'] = clean_custadd['state'].replace('Victoria' ,'VIC')

In [104]:
clean_custadd['state'] = clean_custadd['state'].replace('New South Wales' ,'NSW')

In [105]:
clean_custadd['state'].value_counts()

NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

In [106]:
###Merge tables on customer ID, only retaining data where customer are present in each table (inner join)
cust_list = pd.merge(clean_custdemo,
                     clean_custadd,
                      on ='customer_id', 
                      how ='inner')
cust_list

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,37,6 Meadow Vale Court,2153,NSW,Australia,10
2,9,Mala,Lind,Female,97,1973-03-10,Business Systems Development Analyst,Argiculture,Affluent Customer,N,Yes,8.0,44,97736 7th Trail,2023,NSW,Australia,12
3,12,Sawyere,Flattman,Male,58,1994-07-21,Nuclear Power Engineer,Manufacturing,Mass Customer,N,No,8.0,23,44339 Golden Leaf Alley,4557,QLD,Australia,4
4,13,Gabriele,Norcross,Male,38,1955-02-15,Developer I,Financial Services,High Net Worth,N,Yes,8.0,62,2 Sutherland Street,3799,VIC,Australia,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,3993,Andi,Dumelow,Female,6,1974-12-05,Librarian,Entertainment,Mass Customer,N,No,10.0,43,681 Elmside Place,3750,VIC,Australia,6
2624,3994,Stephie,Byars,Female,5,1989-04-07,Structural Analysis Engineer,Manufacturing,Affluent Customer,N,No,12.0,28,2918 Summer Ridge Hill,3030,VIC,Australia,2
2625,3995,Rusty,Iapico,Male,93,1975-12-12,Staff Scientist,Manufacturing,Mass Customer,N,Yes,14.0,42,613 Erie Lane,2088,NSW,Australia,12
2626,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0,42,0 Transport Center,3977,VIC,Australia,6


In [107]:
###Join transactions to joined demographic and address data
###Definitive Cust List
cust_list = pd.merge(cust_list,
                     clean_transac,
                      on ='customer_id', 
                      how ='inner')
cust_list

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,country,property_valuation,transaction_id,product_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,94,86,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,3765,38,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,5157,47,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,9785,72,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,13424,2,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,3496,Danya,Burnyeat,Male,99,1986-04-25,Editor,Manufacturing,Mass Customer,N,Yes,19.0,31,2565 Caliangt Point,2171,NSW,Australia,9,9769,5,2017-03-07,0.0,Approved,Trek Bicycles,Mountain,low,medium,574.64,459.71,2003-02-07
12966,3496,Danya,Burnyeat,Male,99,1986-04-25,Editor,Manufacturing,Mass Customer,N,Yes,19.0,31,2565 Caliangt Point,2171,NSW,Australia,9,18797,16,2017-04-02,1.0,Approved,Norco Bicycles,Standard,high,small,1661.92,1479.11,2010-05-05
12967,3497,Thia,O'Day,Female,73,1986-05-03,Administrative Assistant IV,Manufacturing,Affluent Customer,N,Yes,18.0,31,96 Delladonna Trail,3976,VIC,Australia,5,6926,1,2017-11-08,1.0,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,2016-11-14
12968,3497,Thia,O'Day,Female,73,1986-05-03,Administrative Assistant IV,Manufacturing,Affluent Customer,N,Yes,18.0,31,96 Delladonna Trail,3976,VIC,Australia,5,8276,18,2017-09-01,1.0,Approved,Solex,Standard,medium,medium,575.27,431.45,1996-04-05


In [108]:
clean_custdemo.head()


,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,37
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,63
8,9,Mala,Lind,Female,97,1973-03-10,Business Systems Development Analyst,Argiculture,Affluent Customer,N,Yes,8.0,44
9,10,Fiorenze,Birdall,Female,49,1988-10-11,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,20.0,29


In [109]:
clean_custadd.head()


,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,NSW,Australia,10
1,2,6 Meadow Vale Court,2153,NSW,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,NSW,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [110]:
clean_transac.head()
clean_transac.shape

(19445, 13)

In [111]:
cust_list.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,country,property_valuation,transaction_id,product_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,94,86,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,3765,38,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,5157,47,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,9785,72,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,13424,2,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09


In [112]:
final_custdemo = cust_list
final_custdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,country,property_valuation,transaction_id,product_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,94,86,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,3765,38,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,5157,47,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,9785,72,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,13424,2,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09


In [113]:
###Get customer ID and address fields only
final_custdemo = final_custdemo.iloc[:, :13]

In [114]:
###Drop any duplications following join
final_custdemo = final_custdemo.drop_duplicates()
final_custdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64
11,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,37
14,9,Mala,Lind,Female,97,1973-03-10,Business Systems Development Analyst,Argiculture,Affluent Customer,N,Yes,8.0,44
20,12,Sawyere,Flattman,Male,58,1994-07-21,Nuclear Power Engineer,Manufacturing,Mass Customer,N,No,8.0,23
27,13,Gabriele,Norcross,Male,38,1955-02-15,Developer I,Financial Services,High Net Worth,N,Yes,8.0,62


In [115]:
final_custdemo.shape

(2327, 13)

In [116]:
final_custadd = cust_list
final_custadd.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,country,property_valuation,transaction_id,product_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,94,86,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,3765,38,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,5157,47,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,9785,72,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,13424,2,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09


In [117]:
###Get customer ID and address records only
final_custadd = pd.concat([final_custadd.iloc[:,0],final_custadd.iloc[:,13:18]],axis = 1)

In [118]:
###Drop any duplications following join
final_custadd = final_custadd.drop_duplicates()
final_custadd.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,NSW,Australia,10
11,2,6 Meadow Vale Court,2153,NSW,Australia,10
14,9,97736 7th Trail,2023,NSW,Australia,12
20,12,44339 Golden Leaf Alley,4557,QLD,Australia,4
27,13,2 Sutherland Street,3799,VIC,Australia,6


In [119]:
final_custadd.shape

(2327, 6)

In [120]:
final_transac = cust_list
final_transac.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,country,property_valuation,transaction_id,product_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,94,86,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,3765,38,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,5157,47,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,9785,72,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64,060 Morning Avenue,2016,NSW,Australia,10,13424,2,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09


In [121]:
###Get customer ID and transaction records only
final_transac = pd.concat([final_transac.iloc[:,0],final_transac.iloc[:,19:]],axis = 1)

In [122]:
final_transac.head()

,customer_id,product_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_first_sold_date
0,1,86,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,38,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,47,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,72,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,2,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09


In [123]:
final_transac.shape

(12970, 12)

In [124]:
writer = pd.ExcelWriter('Sprocket Central Pty Ltd Clean.xlsx', engine='xlsxwriter')

final_custdemo.to_excel(writer, sheet_name='Customer Demographic', index = False)
final_custadd.to_excel(writer, sheet_name='Customer Address', index = False)
final_transac.to_excel(writer, sheet_name='Transactions', index = False)

writer.save()